In [46]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
import time
# import argparse
# from getpass import getpass
# import mechanicalsoup

In [ ]:
path = "./chromedriver-win64/chromedriver.exe"
serv = Service(executable_path=path)

opt = Options()
opt.add_experimental_option("excludeSwitches", ["enable-automation"])
opt.add_experimental_option("useAutomationExtension", False)
opt.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=serv, options=opt)

username = "fxu73@gatech.edu"
password = "[hdph98"

loginurl = 'https://eharvest.acfb.org/Login.aspx'
homeurl = 'https://eharvest.acfb.org/Default.aspx'
invurl = 'https://eharvest.acfb.org/InventoryView.aspx'
driver.get(loginurl)

try:
    WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.ID, 'txtUserName')))

    u_input = driver.find_element(By.ID, 'txtUserName')
    u_input.clear()
    u_input.send_keys(username)

    p_input = driver.find_element(By.ID, 'txtPassword')
    driver.execute_script(f"arguments[0].value='{password}';", p_input)

    login_button = driver.find_element(By.ID, 'btnLogin')
    login_button.click()

    WebDriverWait(driver, 3).until(ec.url_to_be(homeurl))

    inv_button = driver.find_element(By.ID, 'mnuMain_btnInventory')
    inv_button.click()

    WebDriverWait(driver, 3).until(ec.url_to_be(invurl))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # print(html)
    headers = soup.find("div", {"id":"grdData_GridHeader", "class":"rgHeaderDiv"}).find("thead").find("tr").find_all("th")
    headers = [h.get_text(strip=True) for h in headers]
    
    row_tags = soup.find("div", {"id":"grdData_GridData", "class":"rgDataDiv"}).find("tbody").find_all("tr")

    data = []
    temp_cat = ""
    for row in row_tags:
        cols = row.find_all('td')
        curr = [col.get_text(strip=True) for col in cols]
        if curr[0] != "":
            temp_cat = curr[0]
        else:
            curr[0] = temp_cat
        data.append(curr)
    df = pd.DataFrame(data, columns = headers)

    print("Login success! Now on the inventory page.")
except Exception as e:
    print(f"Error during login: {e}")
finally:
    time.sleep(0)
    driver.quit()

df

Login success! Now on the inventory page.


,Product Category,Item #,Description,Pkg. Info,Storage,Qty Avail,Qty Limit,Qty Min,$,Cs/Pallet
0,Baking Products,124667,"Pizza dough 16""","18/ 16"" pizza dough",FR,51,0,3.0000,2.64,0
1,Beverages,08599,Sweet Tea Mio,12/1.62 fl oz bottles per cs,DR,20,0,0,0.24,0
2,Beverages,1122121,Sparkling Blk Mango UnSw-Tea,"8/24pk, 20 lbs eaxh",DR,6,0,0,1.60,90
3,Beverages,866484,Assorted Beverage Pallet,"SparklingUnsweetTea,Diet Cran",DR,1,0,0,87.84,0
4,Beverages,96682422,Pepsi (Pallet),20 fl oz Bottles (pallet,DR,7,0,1.0000,160.00,0
...,...,...,...,...,...,...,...,...,...,...
125,Snacks,25806,Assorted Chips,ORDER BY PALLET,DR,1,0,0,22.48,1
126,Snacks,265546,Remix Chex Mix,8/4.25 oz bags per cs,DR,100,0,0,0.24,156
127,Snacks,46481,Assorted Chips Pallet,"Doritos,Sunchips,Etc",DR,1,0,0,16.24,0
128,Snacks,524596,GingerBread House Kit,6/27oz,DR,"1,231",0,1.0000,0.56,36


In [ ]:
# Next steps: clean df
    # remove things jenny doesn't want
    # check datatypes
    # how will parsing for things go --> recommend certain ones that will be used/are on menu?
    # need an old vs new version to be able to tell new vs old apart
# set up notification system
# can probably put this table in the dashboard

In [ ]:
########################################## EVERYTHING BELOW ME IS GARBAGE #####################################################
#################################### ITS ONLY THERE AS PROOF OF FAITHS SUFFERING ##############################################

# I can't bring myself to delete it yet bc it hurt me so

In [33]:
username = 'fxu73@gatech.edu'
password = '[hdph98'

loginurl = 'https://eharvest.acfb.org/Login.aspx'
homeurl = 'https://eharvest.acfb.org/Default.aspx'

session = requests.Session()
response = session.get(loginurl)
soup = BeautifulSoup(response.content, 'html.parser')

# Hidden fields
viewstate = soup.find('input',{'name':'__VIEWSTATE'})['value']
print(viewstate)
eventvalidation = soup.find('input',{'name':'__EVENTVALIDATION'})['value']
print(eventvalidation)

payload = {
    'txtUserName' : username ,
    'txtPassword' : password ,
    '__VIEWSTATE' : viewstate ,
    '__EVENTVALIDATION' : eventvalidation ,
    '__EVENTTARGET' : 'btnLogin' ,
    '__EVENTARGUMENT' : '',
    '__ASYNCPOST' : 'true' ,
    'btnLoginPanel|btnLogin' : 'LOG IN'
}

login_r = session.post(loginurl, data=payload)
print(login_r.text[:1000])
home_r = session.get(homeurl)
print(home_r.text[:1000])

if home_r.url == homeurl:
    print("Login successful!")
else:
    print("Login failed.")

session.close()

aCzoquK6uecHuZnpYX4zApQmGwgfWXlQfdy2l0ZnaD4BFbHUyv6YTv9w0wJjbqkNTj1BBMw4ATAeQaaVNOwSDGFxaHHq4EugvcNJ2Q6fLZQ+g4kW4KbqOw7zvf68Vi0sfR2TFlUZg7AkR0ECpDVnxvG9DZUWGLbNCls2ydAWUAzgaIgz98+pDynJc/Pdj6BThx24X69Z6wFwM/Xf/0GcIVgpMNc3pFKKvqSPKpSK356LnRn1u6di2JrYgGXAoOmfZa32ZF1xa0yiuzGxCYhLz58fBZ3RAtRbk5UMrtx7aWrYeEs1MV2JT3E61aD+8OKdC4mo0zpfBsF2LmuyUnANx4yKZJyWawSpQt9fhk21InhvcinUqJfkbMLddUt97F157432IyRbOSM3dAxcXSB9uJrYJEp+lAzdd5Qm+srXni48G7Qe2DUeyoSLu/e169yHVT/MQGu3IoAkn3ujAA7NkiJnB3LDrlhZdc7Itm0BrFesfih8bl8Ulrm0lfQ5YjAfkWXvjV1uq+/LnvBcmwejRMe1EGmesnH6HQuT94LWJmHr8SsgUX8XEz8nSGs2LdHuCHPaqOY51AiJXlnAYh+vNAyQQPUYcwnfXcOLK9Ig1Kwh0mzSxaG5qfpKYwtQ/hfc+UCquV/tYFeZAT4MXTlq+Xv+OeFsqJsQ7nvKg26y3XqhRwkWgHw47s7G1axrmtS6FMuQFu8IF69NBW3GwGwSRuLyJ0OnrgCjSZu/uc6wiF4uHQiZ2ihQx8v4XJemAP8MyfWavQGKGZR+0zeNxBTD+44lfaFgg3AYdcfYcD1sIXRFl31vQvjt/AnOgphQD++6XhI05uAMw2zJF8xJIBF8Joc/oQIROimR4gYJBzYeqI3cJJ4SiU7g19NMmFPFADIP4qumuye7mfYF3GTqnh7/psdqdALhKebJA4dZ/WYn4oAy7RPOFwf8R4G0v4Cqd6kj0SgNNqxSm3R1GYjDzb8RCWK9tGWTezxJm6w0eJKo

In [20]:
# resp = requests.get(loginurl)
# print(resp.text)

In [ ]:

loginurl = ('https://eharvest.acfb.org/Login.aspx')
homeurl = ('https://eharvest.acfb.org/Default.aspx')

payload = {
    'txtUserName':username
    , 'txtPassword':password
    # , 'RadScriptManager1':'btnLoginPanel|btnLogin'
    # , 'RadStyleSheetManager1_TSSM':';Telerik.Web.UI, Version=2021.3.914.45, Culture=neutral, PublicKeyToken=121fae78165ba3d4:en-US:bd4f5d20-e2f4-41b1-99ef-02ee4a064af0:45085116:27c5704c:505983de:d7e35272:959c7879:ba1b8630:3e0dfe6c;Telerik.Web.UI.Skins, Version=2021.3.914.45, Culture=neutral, PublicKeyToken=121fae78165ba3d4:en-US:bd68d779-31cd-457f-adc8-06e8ac5469a8:a7b34603:d237931b:a4a302e4'
    # , 'RadScriptManager1_TSM':';;System.Web.Extensions, Version=4.0.0.0, Culture=neutral, PublicKeyToken=31bf3856ad364e35:en-US:18f1b484-bbc5-4e2e-8ca4-477603537f34:ea597d4b:b25378d2;Telerik.Web.UI:en-US:bd4f5d20-e2f4-41b1-99ef-02ee4a064af0:16e4e7cd:f7645509:22a6274a:ed16cbdc:88144a7a:33715776:b7778d6c:24ee1bba:6d43f6d9:874f8ea2:c128760b:b2e06756:19620875:92fe8ea0:fa31b949:4877f69a:f46195d3:490a9d4e:11e117d7'
    # , 'txtUserName_ClientState':'{"enabled":true,"emptyMessage":"Enter Email Address/Username","validationText":"'+ username +'","valueAsString":"'+ username +'","lastSetTextBoxValue":"'+ username +'"}'
    # , 'txtPasswordMask':'Enter Password'
    # , 'txtPasswordMask_ClientState':'{"enabled":true,"emptyMessage":"Enter Password","validationText":"","valueAsString":"","lastSetTextBoxValue":"Enter Password"}'
    # , 'txtPassword_ClientState':'{"enabled":true,"emptyMessage":"","validationText":"'+ password +'","valueAsString":"'+ password +'","lastSetTextBoxValue":"'+ password +'"}'
    # , 'btnLogin_ClientState':'{"text":"LOG IN","value":"","checked":false,"target":"","navigateUrl":"","commandName":"","commandArgument":"","autoPostBack":true,"selectedToggleStateIndex":0,"validationGroup":null,"readOnly":false,"primary":false,"enabled":true}'
    # , 'winForgotPassword$C$txtEmailAddress':''
    # , 'winForgotPassword_C_txtEmailAddress_ClientState':'{"enabled":true,"emptyMessage":"","validationText":"","valueAsString":"","lastSetTextBoxValue":""}'
    # , 'winForgotPassword$C$captchaEmail$CaptchaTextBox':''
    # , 'winForgotPassword_C_captchaEmail_ClientState':''
    # , 'winForgotPassword_C_btnUpdatePassword_ClientState':'{"text":"Reset Password","value":"","checked":false,"target":"","navigateUrl":"","commandName":"","commandArgument":"","autoPostBack":true,"selectedToggleStateIndex":0,"validationGroup":null,"readOnly":false,"primary":false,"enabled":true}'
    # , 'winForgotPassword_ClientState':''
    , '__EVENTTARGET':'btnLogin'
    , '__EVENTARGUMENT':''
    , '__VIEWSTATE':'oVUs6lhXt4gxXeBDNdu4yA80jWK+NF7/MIy0CZSgHCEtm02ISDId6gm2EK/HOiydQj64xACioDl56yLxK8UrpvvxQJrknjRjb04q0zNo+d59hux1bvOND51/OX1U1Fdu1k6yQqJmFgpsJLZNHgA/P4Xntj8GfztBRxkgTRiqFUF9L0troVIeVGwnDMj/GtsXLbVfpZGeGysj6BdvJvg/7S8FkOcuX+wFsMb7V3pAHdxKb7WPFuwVRSOJyXLbGEyTPKAQJdN3sTTHz/udW7UYbIP2ROKCnq4b1XXZSbMUQMRTqj8rLwb6o18uYk3qHeOZxEXUvqOQMmFbNcUtzMJ/LyGKLBTE0UJ/PXQJ95HUGWo1AyjGPU9p/J0VYkiVOxnTJA5tWtrLy+vLTdvdR5iyQCaA1C00wBuSvGPB/ilA/4gODbpZTM4GW42Sq4u9z9eM78UP0Suig+t8OKXtNMMjixwh/Cmxc9jQR++uYtD1rQ75rf+nVWexQmD596mg5ZVfy1zZn7tIanQaFNNfzmn/Gj6kVaj2Hkfhrv1FwNFUE1T9mj/4nsae5gnaXA/ZHshwM8BMPl/IRyqO9k7nlSg4oOORhmU6EsA9bxQpYr/m5frnOr+47BO52665DoUXCh8VGF706+nt4f72AQojFXzz3c96tLg4KI4NMsNUk1FZ1mto4R41k9cO11Er3xeL4B7k7O4sSh7gBmmaZOzKmpdd/DJ/8wLp9Apvd3f06SrhgcBqxq49sDYU/jHi2xkPF+Lg15ECxvJQVtZwiGu9/mgoKQVgxVKS3sXZvlLHkAoxBsDoRWrCJP0ifSvn5EEr8lr6DXqYWYoiIXDj1na9rhwYR9mgwu/j4rAD6P9I1mUz+svMnwTjQiy4HEckobj7bR5umasq+2Y7F5LqVKNQjdu06HWLVfYfCXnigRYrBUhLWTc57jAvzSAhSNzYMXa9oAy5Ie1zao0mgH2UG1IRCglyHdpJjtwLf9Q7y9toqbV102pR3SR84HONpDk6qtMg8fNOipd0m0j8wCUzlZzQCpwTbvvXEBCz01ySj5AbtKxRvrXD6jbgMVOO0HXZg0LkmByJORIk2GozoW5edHTE31vBY2O0sFhgdBwJAe5aKGFvsgyMuqsYnjClLNJe5eD/lFPU0BgWO/8Nlw51BqzdhxMuGCgQuqS2HH8gWBASTwcYq/28KY0U5SocwwzcEXj7CvVWeMRfY3DgOvkqfK31aNyO5d/iYbkYatnE0guz5yIdBN8V+IN9JD8QQS5Xwu4WutB98ii4KWw0WDlbIOYBbIG7q8iIIeaOWhyTuJIXAN7Eakrmx1lps6roT//DT4IQQ0dO3to/snbNvcN1L4LwHrUWGxwFCJaX6L68vrVyGEqe4SAgDSKSY8d/kgWFGs8lJGivTt1Ty5xcOqPh5pLeNoQJnGJWZ7C/JkKfI+DbSjyh8Hxr0ZSeuJqssuswTzJRFwk/ertvpUUhP41rGLSlWfEaIs/BO7TZUItqYU7iOX0rBe/xzf+L5qKZTjaln99fK0/eSH2ugOuIG6tsI2aHwey/DxEDSRH2kkZsl+hMs+Sq8uMQ7u7AnXZvwz15nVMrgR3ZPh1uc5Ks/WAd32/EHDZbmUQHvuESabdnND2vpr+nDfKgjkOWHcB5lTON4sUNZNFziw2WUW8tSG457EAPj74dzUPCTXjDrQ2p1pKthnGCHH4OUPV50u8THfRNNvBFEqdOSwm8PgL+AnpsXBvSC2p07OqfB6D8+IDU0Da6u/S9Ggp7WnU6T+nj7xDsPyUuF3F0KOcn8Dm/0ae82RWCswENyxsq1AWPXgKGmT0eqSWsLVnanD2koDoZ6bWatRxNZEq7LlOisjEZWHpDF3DP+U9oKZ9ShmuELBomy/m09zwe/aa9q4hhF092iTPuT6GIjAjOTRy0o2tSNik1SzsPCj25o4D4keNihtmOpzjGQhiYVWeWtPHhFkSbjlj3da0GZnF6aod243arX0QVv53P2KEwp5gZDeaNaC5PqsFkaetL3gRYcsXA7nj1R0uFPlAAQr/ms12tAYG34htkg+DBQvpGiclZH6oGztaCNjRX+2WKH/QsPqJLClaz3v9pCNY/EECjO+rY07WthpX/SOhHdgUChv/dBGEsS0h4hA4VBtnOQ221kGiZdfu+EhpbwF6ZCvPRuf7fDgW6MvP8E12+RiJkkXrXQLbVfDjTqebpfLR1jNGWepRgWOODfonRzkpC/wGwd2eJowGjvdV61z8FjW4v4RFWUBOhwt7lTewgxjwhDwyv4Gsrc/SYUSUHZcgW+iNq/B9gKVvvWACCe91GwvhcyU4XGuyc1REfzAboHgpDzLoHaK7ImKAMfUt18v4MbktxPenZElvwjzoSF3fOtmJH0pbGyaaJF91UG18eVKzz4s24kPFcM6CgLrsy20pf78gNH7oKY1jSU52TO+3fxvkeTcbNn1ZtgBoysS8Yr2PnfJprzw7OkA0hoanRFPWdNAPIkT13fIsCGOOyUEoFbrijzVci3PfzgZ8N8bpjPNwhxUNCX2CbIe6YBHd07gPnn1czkFa9ZDwCbmKxXXtnEkh0nd6f6JvY/fyvJuCU8EXWSTARBXPW+0jGVPun2+qq46170oN/mFEkzc5QWu7k92JuEbZX7v7JEgtbhw4F63o2vbh9QuO7uOVuQwpwHWA2r4mLokN5ard3xVtmB3gJ8sQ4/mVyyGR+ERANfj1yFTAVAw9V7CJBCAe0hRmrj8ISMZZNcVvchU4lxIeg/xhelH3BC9K2HvxyW6n97FC6LrzqvAUBkFZIBEuypWsigaHWOU9v8C8Hd/Hwxp+WNSVaB6PO3D/i0ugCBdGId1do93VpyjmrGSb2U71QeHGrQh1XfcRtaYNPmL+Ju/TF3+GRtdJslUP4pRzi2Ywhs58It5sIfAOjOEr+EZidGMrz4OdKQSRTcn+BglvpTQ/E1qoPAiRPHbHs2ZV29A9B2EjI6dRnhL0VIgl+X5Pg2oTWEK19CdhVMhBvm6FdAlYM8lQK82ocTBqeqIz5g0eUQco7wB35UJc39XOK04MzAIII1PQttgAnHWJqnFKP/q4TMvwS0G9SfEurcmGEEzMYEmyITSzVkUXp7GwOdcjnxatzqYYB+qg85KofXlQM2D8Ibnx1eCod5o3AerD/FF7LkXLHn6il6MYRqAeZW17tiFrwsxQcvaV7jMDAQXm+BUHVgV6IHVFwjXcRUPkeTDJDKh/tZFfTGQ+fb3KgstCUR3PqG52LASvzfjORUe8wpi9ZKTCsNWM6K1w2oxiPoGyj+Ezb6oQ22Pg7Q1iE4hPnJ67TlQJMxFOFd39XBn0iTNNkrFeBno9fky56zbiBfOy80RhSmgQ2wHzjd1tSwVg06dkGBsbFO0+BTR4yl3uXjnivfR6DRVdggBFuZLGpeXF1MXU6Q0vlzHlZpyWTBU3pguEI+StpMfuKdqwoPGA1fH3xVYk2kHjZMCHY/0BEbr2TPs3DRTbonXRnGp+CEiARL40hYSoZYH8kTYJESnj76fIeMZ8LANpkmRt3ew5LI8GTYik7rN3aPOulpuOwn5b67/OwLA/mTqe1aAiF8s79v7lojkSGry/lGuq0SJCgz2qQHoiLFaRFvLCxywa26pVRgHW1LU1CtVpdL9WgSQGxiIK9f66U2FFC9XgmGLJnOe0BNd2w3/YJ/FtUQ+pjPwWY89+4EHSW0nLviySopAVjbgRoCc7DClBBVmBTHSTaXNz89xwhsYOk6tagjRx5nWEk1IQ0Q6Fui42EzQXI7NGpesmH/IfwikdH/SEsjolO7RCeYIG3Iam0Atn+1+9/8F1YpMA9bgKBTCUuZh+BxelY3NXt+GRBOpkFtDh5EQTpONf9+zwSoRMr80QVJIE+AeVUpGfxSl6lvVegTU5Q4aUybyCjXZP8GTUGvBn438On3hwqx7lRcj80M0JTMnwWOQbTYH2q4RlesmbXWGnQLuANe20zK+kWHUXUbQs5P9TR6dy/ZPEaRighi9gPdI2CWbydzGBTAx8CjWEeaBVSNCNh3sQMdxTIDDlC3PjrX4V3RuGAO/RdEdOyn03s1orRewi1mv0TX51Za/xU60y8ar18ZaERmhjBZTNDmq/e8dH354Zc2dL7UvDCcLjdGagdavfzsjHnYGk8G2wWEpMpaoD/DlNjMai+kuHTrz7MRFKkSYJS3r2IAtLKk8LxDOLRfOrmBWWlmEJ+2MU2YZ2ecfkm3HK0WZUE+vxWOZs/fbv0iaz5kbtZTHYvxJ8pc5sv4LRMPdRxBzLV56UP2gHZ9jCZ/mfn+uE6On4andUlr15GmvXJhu2vKs1NQ+b4g4nwNiYX6RWrhWMZJ4tq8ekRSM66n8CDAcI/QwNMsU+WEZDzoiQYzBNYfAeUjseeivT4T+2UW2K2PA3PiNJHjvyrPF+LkZs3medLzByk3ZMEMFTF03t3UtBekRShG7ipnw9231gB1zGwkZRNO10VBtfsxbZCu0CwU2BCmc8IzFTYZ+dVygJ6kjS5k95IcOOW64iHdeSltLc4MUO5IvubxsLERJYN1eccWwvwOOhferI+cLzgZWvEZgyFe4/Mta4kQV3vleK7HH8Mjyudk8ek3V+ACUlSEnS3Ar0IiaxcLhQjGPQGLF1JqvZk6H2hZnNWvZ37tNgTaL0Ukxi3C3QxuxO7rVUr0+0CBTnvt5kx+1s/sOjZApXMOpDAPrwPdsjGbPam5sA6zvfREB+Q3RhuBn14mx3qZ8vQ9LJbNeOOe79Rd4JQWCvN+b1sEmOXYMZbUJzTGeiDIyP6CTwiR9yCpnXdTcb2eVHEa5TF5qG0JAQUhezpFR0f9Fc+VtXmDSbJnDPszPk2A7b8M0KtbVZrhKHU/9fPPhrdV5w2Z9wtsL9EsInXh4FyUiQTmug4bjBXIFgBLMblp5FIYIrqyLozES8FrVI69nqQuQRT995K5yufAGiNVix11Zu1Gb4GCghkZFqvNKQN/kGcXshgio2dH9kYpk2E0MVMxT3UhP4tqOtg40I3SO2IEssMcC5bZNyS3MTCqRnNIJn5CcvXahlDc3J2VFAcExjVL7iNT0tw1m1mlJhYfbRD7fxJjuHq7fVoklIJr5qP4Q2BGNgiXLXFSnBzN5CMcSYnQOxBW19P1dsnV8gK7fb+RuL/cNceNWf7Lz6EbyyyBWW4YfkJDAp3jHn7C4DOo8L/9gDEXpxKn2O//oM7h0Ep03ipWNwmVk3K6zdov3Z8hTEoQEks3qp+Zmcollei7A8CjfR4WuZBmAdoe6I0CN4huysDkBSnV0wVtoqYFjyyhSRTif/PaYPTUVnKMAYJlrsg3mREGR+YyREB2jMrkEv1ZgIGLm75bGm+kKtIgRQdmkQxnj0dhCZh'
    , '__VIEWSTATEGENERATOR':'C2EE9ABB'
    , '__EVENTVALIDATION':'qoEwJNf8uwUhQeJbd/eqlRacUGGq/+v1OrwT4nsKQo5SG/0wiitKnfb9eNMEbUtSJjOGcervfzV3LryEClVYh4HZdsI40kokmkO1LK5UyweYWbEwX6UwjA1KZYBMbVqqSQAobemWqnDgZIsO5Qo5Rg5jy0silAPImztdX9+W1cKe5EtL2UhW5gOeBYqoDlEVLdwxvI7v1a/iSpovallzS2U5ogcdXp4Q64PxIvHGAWDUYd91xvDySMUqoCyaXZR1VcNCnC+X3jlkKYCCPo38z+wbUndFC0sf8e/zr9OAadfiYFBW04bkKY4RyNXl5aW4KeVO5PAegEvw7HeBdejF89bzDBewKs8b5rai7iuOVP4='
    , '__ASYNCPOST':'true'
    # , 'RadAJAXControlID':'RadAjaxManager1'
}

In [22]:
r = requests.post(loginurl, data=payload)

print(r.text)

1|#||4|17|pageRedirect||%2fErrorPage.aspx|


In [ ]:
r2 = requests.get(homeurl)

In [ ]:
print(r2.text)

In [10]:
browser = mechanicalsoup.StatefulBrowser()

In [ ]:
browser.open()
# print(browser.get_url())

<Response [200]>

In [ ]:
browser.select_form('form[action="./Login.aspx"]')
browser[""]

In [3]:
parser = argparse.ArgumentParser(description="Login to ACFB")

In [9]:
parser.add_argument("Email Address/Username*")

_StoreAction(option_strings=[], dest='Email Address/Username*', nargs=None, const=None, default=None, type=None, choices=None, required=True, help=None, metavar=None)

In [8]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] username
ipykernel_launcher.py: error: the following arguments are required: username


SystemExit: 2

In [41]:


# The login URL you want to target
login_url = "https://eharvest.acfb.org/Login.aspx"

In [ ]:

r = session.get(login_url)


In [44]:
soup = BeautifulSoup(r.text, "html.parser")

In [46]:
login_data = {
    'username':'fxu73@gatech.edu',
    'password':'[hdph98'
}

In [47]:
login_response = session.post(login_url, data=login_data)

In [48]:
if login_response.ok:
    print("Login successful")
else:
    print("Login unsuccessful")

Login successful


In [49]:
print(login_response.text)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	ACFB - eHarvest Login
</title><link href="styles/PTChild.css" rel="stylesheet" />
    <style type="text/css">
    </style>
<link href="/Telerik.Web.UI.WebResource.axd?d=PMrIT5dOWaVYIcpFWUE4nKsbqQQ0_xaFbNdTtlTcy77PbaebZaDjuFVxPThkzDg8xBpwJGrYBlXhRQtESW2XZkuPnSev6tmxOektaTA_OxEW6_WBeJVAHPwBndp0DPvx0&amp;t=638528015272927808&amp;compress=1&amp;_TSM_CombinedScripts_=%3b%3bTelerik.Web.UI%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd4f5d20-e2f4-41b1-99ef-02ee4a064af0%3a45085116%3a27c5704c%3a505983de%3ad7e35272%3a959c7879%3aba1b8630%3a3e0dfe6c%3bTelerik.Web.UI.Skins%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd68d779-31cd-457f-adc8-06e8ac5469a8%3aa7b34603%3ad237931b%3aa4a302e4" type="text/css" rel="stylesheet" /

In [45]:
csrf_token = soup.find('input',{'name':'csrf_token'})['value']

TypeError: 'NoneType' object is not subscriptable

In [5]:
print(r.text[:1000])


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	ACFB - eHarvest Login
</title><link href="styles/PTChild.css" rel="stylesheet" />
    <style type="text/css">
    </style>
<link href="/Telerik.Web.UI.WebResource.axd?d=PMrIT5dOWaVYIcpFWUE4nKsbqQQ0_xaFbNdTtlTcy77PbaebZaDjuFVxPThkzDg8xBpwJGrYBlXhRQtESW2XZkuPnSev6tmxOektaTA_OxEW6_WBeJVAHPwBndp0DPvx0&amp;t=638528015272927808&amp;compress=1&amp;_TSM_CombinedScripts_=%3b%3bTelerik.Web.UI%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd4f5d20-e2f4-41b1-99ef-02ee4a064af0%3a45085116%3a27c5704c%3a505983de%3ad7e35272%3a959c7879%3aba1b8630%3a3e0dfe6c%3bTelerik.Web.UI.Skins%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd68d779-31cd-457f-adc8-06e8ac5469a8%3aa7b34603%3ad237931b%3aa4a302e4" type="text/css" rel="style


In [29]:
#crsd
tag = soup.find_all("td")
print(tag)

[<td>Email Address/Username*</td>, <td>
<span class="RadInput RadInput_Simple" id="txtUserName_wrapper"><input class="riTextBox riEmpty" id="txtUserName" name="txtUserName" size="20" type="text" value="Enter Email Address/Username"/><input id="txtUserName_ClientState" name="txtUserName_ClientState" type="hidden"/></span>
</td>, <td style="height:15px;"></td>, <td>Password*</td>, <td>
<div id="divPasswordMask">
<span class="RadInput RadInput_Simple" id="txtPasswordMask_wrapper"><input class="riTextBox riEmpty" id="txtPasswordMask" name="txtPasswordMask" size="20" type="text" value="Enter Password"/><input id="txtPasswordMask_ClientState" name="txtPasswordMask_ClientState" type="hidden"/></span>
</div>
<div id="divPassword" style="display: none;">
<span class="RadInput RadInput_Simple" id="txtPassword_wrapper"><input class="riTextBox riEnabled" id="txtPassword" name="txtPassword" size="20" type="password"/><input id="txtPassword_ClientState" name="txtPassword_ClientState" type="hidden"/>

In [26]:
print(tag.text)

In [31]:

# Step 2: Define the login payload (your email and password)
# You may need to inspect the form fields in the HTML to get the exact names
payload = {
    'Email': 'fxu73@gatech.edu',
    'Password': '[hdph98',
    # 'csrf_token': csrf_token,  # Add this if necessary
    # Any other hidden fields required by the form
}

In [35]:


# Step 3: Post the login request with the payload
login_response = session.post(login_url, data=payload)

In [39]:
print(login_response.text)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	ACFB - eHarvest Login
</title><link href="styles/PTChild.css" rel="stylesheet" />
    <style type="text/css">
    </style>
<link href="/Telerik.Web.UI.WebResource.axd?d=PMrIT5dOWaVYIcpFWUE4nKsbqQQ0_xaFbNdTtlTcy77PbaebZaDjuFVxPThkzDg8xBpwJGrYBlXhRQtESW2XZkuPnSev6tmxOektaTA_OxEW6_WBeJVAHPwBndp0DPvx0&amp;t=638528015272927808&amp;compress=1&amp;_TSM_CombinedScripts_=%3b%3bTelerik.Web.UI%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd4f5d20-e2f4-41b1-99ef-02ee4a064af0%3a45085116%3a27c5704c%3a505983de%3ad7e35272%3a959c7879%3aba1b8630%3a3e0dfe6c%3bTelerik.Web.UI.Skins%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd68d779-31cd-457f-adc8-06e8ac5469a8%3aa7b34603%3ad237931b%3aa4a302e4" type="text/css" rel="stylesheet" /

In [36]:


# Check if login was successful by examining the login_response content or status code
if "Dashboard" in login_response.text:
    print("Login successful!")
else:
    print("Login failed.")

Login failed.


In [50]:

session.close()
# Step 4: Now you can use the session object to make further requests while logged in


In [2]:


cookies = {
    '_hjSessionUser_2020664': 'eyJpZCI6IjVkMjY0MTA5LTc5M2QtNTMxNC05MzNjLTA1Y2Q2NTU5ZWZkMiIsImNyZWF0ZWQiOjE3MzE0NjA5NDY2MTcsImV4aXN0aW5nIjp0cnVlfQ==',
    'ASP.NET_SessionId': 'zx4ogmkcn3aou4yts2pgwzkx',
}

headers = {
    'authority': 'eharvest.acfb.org',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': '_hjSessionUser_2020664=eyJpZCI6IjVkMjY0MTA5LTc5M2QtNTMxNC05MzNjLTA1Y2Q2NTU5ZWZkMiIsImNyZWF0ZWQiOjE3MzE0NjA5NDY2MTcsImV4aXN0aW5nIjp0cnVlfQ==; ASP.NET_SessionId=zx4ogmkcn3aou4yts2pgwzkx',
    'referer': 'https://eharvest.acfb.org/Order.aspx?o=372373',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

url = 'https://eharvest.acfb.org/InventoryView.aspx'

r = requests.get(url, cookies=cookies, headers=headers)

In [3]:
pprint(r.text)

('\r\n'
 '<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" '
 '"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n'
 '<html xmlns="http://www.w3.org/1999/xhtml">\r\n'
 '<head><title>\r\n'
 '\tACFB - eHarvest Login\r\n'
 '</title><link href="styles/PTChild.css" rel="stylesheet" />\r\n'
 '    <style type="text/css">\r\n'
 '    </style>\r\n'
 '<link '
 'href="/Telerik.Web.UI.WebResource.axd?d=PMrIT5dOWaVYIcpFWUE4nKsbqQQ0_xaFbNdTtlTcy77PbaebZaDjuFVxPThkzDg8xBpwJGrYBlXhRQtESW2XZkuPnSev6tmxOektaTA_OxEW6_WBeJVAHPwBndp0DPvx0&amp;t=638528015272927808&amp;compress=1&amp;_TSM_CombinedScripts_=%3b%3bTelerik.Web.UI%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd4f5d20-e2f4-41b1-99ef-02ee4a064af0%3a45085116%3a27c5704c%3a505983de%3ad7e35272%3a959c7879%3aba1b8630%3a3e0dfe6c%3bTelerik.Web.UI.Skins%2c+Version%3d2021.3.914.45%2c+Culture%3dneutral%2c+PublicKeyToken%3d121fae78165ba3d4%3aen-US%3abd68d779-31cd-457f-adc8-06e8ac5469a8%3

In [11]:
Soup = BeautifulSoup(r.text, "html.parser")